# Business Cycle - Bonds

## Imports

In [25]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v4 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Data Cleaning

In [26]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_93661/2677495952.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

In [27]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data /= 100
display(famafrench_data)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_93661/2041547973.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.000550,0.000188,0.000185,-0.000184,0.000189,0.000745,0.000695
1980-02-29,-0.000123,-0.000162,0.000059,-0.000095,0.000292,0.000789,-0.000132
1980-03-31,-0.001289,-0.000697,-0.000096,0.000182,-0.000105,-0.000958,-0.001181
1980-04-30,0.000396,0.000105,0.000103,-0.000218,0.000034,-0.000048,0.000574
1980-05-31,0.000526,0.000200,0.000038,0.000043,-0.000063,-0.000118,0.000618
...,...,...,...,...,...,...,...
2025-06-30,0.000486,-0.000002,-0.000160,-0.000320,0.000145,-0.000264,0.000527
2025-07-31,0.000198,-0.000015,-0.000127,-0.000029,-0.000208,-0.000096,0.000184
2025-08-31,0.000185,0.000488,0.000442,-0.000068,0.000207,-0.000354,0.000646


In [28]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)
display(benchmark_data)
benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_TR = benchmark_data[['MSCI World']].pct_change()
benchmark_XR = benchmark_TR.sub(riskfree, axis = 0)

benchmark_TR.columns = ['Benchmark Total Return']
benchmark_XR.columns = ['Benchmark Excess Return']
benchmark_returns = pd.concat([benchmark_TR, benchmark_XR], axis = 1).dropna()

display(benchmark_returns)

,S&P 500,MSCI World
Date,,
1986-12-31,242.1700,NaN
1987-01-30,274.7800,NaN
1987-02-27,285.6377,NaN
1987-03-31,293.8792,NaN
1987-04-30,291.2698,NaN
...,...,...
2025-07-31,14412.5527,8214.1572
2025-08-29,14704.7217,8431.0801
2025-09-30,15240.0381,8705.7139


,Benchmark Total Return,Benchmark Excess Return
1991-01-31,0.034364,0.029164
1991-02-28,0.090450,0.085650
1991-03-31,-0.031331,-0.035731
1991-04-30,0.005910,0.000610
1991-05-31,0.020740,0.016040
...,...,...
2025-06-30,0.043488,0.040088
2025-07-31,0.013121,0.009721
2025-08-31,0.026408,0.022608
2025-09-30,0.032574,0.029274


In [29]:
# --- Load Macro Data ---
CPI_forecasts = pd.read_excel(
    f"{path}/Inflation_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
CPI_forecasts.index = pd.to_datetime(CPI_forecasts.index)
CPI_forecasts.index = CPI_forecasts.index + pd.offsets.MonthEnd(0)
CPI_forecasts *= 100

display(CPI_forecasts)

RGDP_forecasts = pd.read_excel(
    f"{path}/RGDP_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
RGDP_forecasts.index = pd.to_datetime(RGDP_forecasts.index)
RGDP_forecasts.index = RGDP_forecasts.index + pd.offsets.MonthEnd(0)
RGDP_forecasts *= 100

display(RGDP_forecasts)

,UK,CH,JP,AU,EU,US
Date,,,,,,
1970-01-31,4.95739,2.31338,7.75127,NaN,NaN,NaN
1970-02-28,4.93065,2.12360,7.75127,NaN,NaN,NaN
1970-03-31,5.14198,2.49918,7.71710,NaN,NaN,NaN
1970-04-30,5.61884,2.59281,7.68362,NaN,NaN,NaN
1970-05-31,6.08365,3.13683,7.24558,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,4.10000,0.10000,3.30000,2.1,1.96760,3.17
2025-07-31,4.20000,0.20000,3.10000,3.2,2.00791,2.79
2025-08-31,4.10000,0.20000,2.70000,3.2,2.02889,2.79


,AU,UK,CH,JP,EU,US
Date,,,,,,
1970-07-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-08-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-09-30,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-10-31,5.26317,NaN,NaN,NaN,NaN,2.994250
1970-11-30,5.26317,NaN,NaN,NaN,NaN,2.994250
...,...,...,...,...,...,...
2025-06-30,2.10000,2.5,2.439516,1.976375,1.522007,1.345350
2025-07-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175
2025-08-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175


In [30]:
# --- Load Bond Futures ---
bond_futures = pd.read_excel(
    f"{path}/Bond Futures.xlsx",
    index_col = 0,
    parse_dates = True
)
bond_futures.index = pd.to_datetime(bond_futures.index)
bond_futures.index = bond_futures.index + pd.offsets.MonthEnd(0)

display("Bond Futures:")
display(bond_futures)

'Bond Futures:'

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,72.58774,89.88,83.91110,58.968750,90.12,NaN,89.28422
1998-02-28,73.84924,90.45,83.86650,58.437500,91.37,NaN,89.20823
1998-03-31,72.76451,90.75,84.08059,58.281250,90.54,NaN,92.70965
1998-04-30,74.66254,91.56,83.99135,58.250000,89.35,NaN,92.89550
1998-05-31,75.80680,93.14,84.46878,58.593750,91.27,NaN,91.67311
...,...,...,...,...,...,...,...
2025-07-31,147.00613,137.03,61.55103,111.078125,196.99,15.03735,121.79588
2025-08-31,150.42250,136.54,62.63541,112.468750,200.93,15.12679,122.26090
2025-09-30,151.15975,135.79,63.31735,112.484375,205.32,15.14941,122.26156


# Global Variables

In [31]:
frequency = 1
t_cost = 0
# window = 12*20
short = True
beta_neutral = False
target_vol = 0.10
rf = riskfree
benchmark = benchmark_returns

## Signal Generation

In [32]:
# --- Compute Business Cycle Signal ---
CPI_component = CPI_forecasts.diff(12)
RGDP_component = RGDP_forecasts.diff(12)

display(CPI_component)
display(RGDP_component)

# # Reindex RGDP trend to match CPI trend index exactly
# RGDP_component = RGDP_trend.reindex(CPI_trend.index).ffill()
# CPI_component = CPI_trend

,UK,CH,JP,AU,EU,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.31793,-1.22807,0.5,-1.71445,-0.54354,0.39
2025-07-31,1.17674,-1.09059,0.3,0.39146,-0.57075,0.39
2025-08-31,1.00881,-0.86037,-0.3,0.39146,-0.12434,0.39


,AU,UK,CH,JP,EU,US
Date,,,,,,
1970-07-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-08-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-09-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-10-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-11-30,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.71445,0.7,2.356965,2.957250,0.993241,-0.529850
2025-07-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325
2025-08-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325


In [33]:
# --- Composite Signal Construction ---
# Logic: For Bonds, we want Low Growth (-) and Low Inflation (-)
# We combine them: 50% Inverse Growth, 50% Inverse Inflation
business_cyle_signal = - (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal = business_cyle_signal.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal)

'Business Cycle Signal:'

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.71445,-0.564448,-0.224851,-1.728625,-1.008965,0.069925
2025-07-31,-0.39146,0.867678,0.092139,-0.488158,-1.438370,-0.026338
2025-08-31,-0.39146,0.752568,-0.131066,-0.188158,-1.354405,-0.026338


## Asset Returns

In [34]:
# --- Bond Returns ---
bond_XR = bond_futures.pct_change()
bond_XR = bond_XR.resample('ME').last()

display(bond_XR)

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,NaN,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,NaN,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,NaN,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,NaN,-0.013159
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


## Portfolio Construction

In [35]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

rank = business_cyle_signal.rank(axis = 1, method = 'average', ascending = False)
display(rank)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0,4.0,3.0,6.0,5.0,2.0
2025-07-31,4.0,1.0,2.0,5.0,6.0,3.0
2025-08-31,5.0,1.0,3.0,4.0,6.0,2.0


In [36]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

rank_mean = rank.mean(axis = 1)
rank_stds = rank.std(axis = 1)
standardized_weights = rank.sub(rank_mean, axis = 0).div(rank_stds, axis = 0)

display(standardized_weights)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.336306,0.267261,-0.267261,1.336306,0.801784,-0.801784
2025-07-31,0.267261,-1.336306,-0.801784,0.801784,1.336306,-0.267261
2025-08-31,0.801784,-1.336306,-0.267261,0.267261,1.336306,-0.801784


In [37]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret = (standardized_weights.shift(1) * bond_XR).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol = strategy_raw_ret.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor = target_vol / expected_vol.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights = standardized_weights.mul(lev_factor, axis=0).fillna(0).loc['1998-03-31':'2025-10-31']

display(final_strategy_weights)

,AU,CH,EU,JP,UK,US
1998-03-31,-10.507564,10.507564,5.253782,-5.253782,0.000000,0.000000
1998-04-30,3.197838,6.395677,0.000000,-6.395677,0.000000,-3.197838
1998-05-31,3.173357,6.346715,0.000000,-6.346715,0.000000,-3.173357
1998-06-30,3.133846,6.267692,0.000000,-6.267692,0.000000,-3.133846
1998-07-31,5.483851,2.741926,0.000000,-5.483851,0.000000,-2.741926
...,...,...,...,...,...,...
2025-06-30,-0.821275,0.164255,-0.164255,0.821275,0.492765,-0.492765
2025-07-31,0.164631,-0.823155,-0.493893,0.493893,0.823155,-0.164631
2025-08-31,0.499222,-0.832037,-0.166407,0.166407,0.832037,-0.499222
2025-09-30,-0.167006,-0.835031,0.167006,0.501019,0.835031,-0.501019


In [46]:
bond_XR.dropna()

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
2015-04-30,0.031739,0.004575,0.033122,-0.004105,-0.009722,0.024468,0.010578
2015-05-31,-0.029421,-0.001763,-0.031644,-0.000515,0.013757,-0.030201,0.000529
2015-06-30,-0.002276,-0.002281,0.004339,-0.010568,-0.010606,-0.002995,0.012103
2015-07-31,0.002128,0.003909,-0.048750,0.010030,0.015314,0.006628,0.007121
2015-08-31,0.009628,0.002791,-0.025527,0.001677,0.004984,-0.012514,-0.008723
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


# Backtest

In [38]:
benchmark_TR = benchmark_TR.squeeze()
benchmark_XR = benchmark_XR.squeeze()

In [43]:
final_strategy_weights.head()

,AU,CH,EU,JP,UK,US
1998-03-31,-10.507564,10.507564,5.253782,-5.253782,0.0,0.000000
1998-04-30,3.197838,6.395677,0.000000,-6.395677,0.0,-3.197838
1998-05-31,3.173357,6.346715,0.000000,-6.346715,0.0,-3.173357
1998-06-30,3.133846,6.267692,0.000000,-6.267692,0.0,-3.133846
1998-07-31,5.483851,2.741926,0.000000,-5.483851,0.0,-2.741926


In [41]:
results = pmp.run_cc_strategy(
    weights = final_strategy_weights,
    returns = bond_XR,
    rf = rf,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark_TR,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,
1998-04-30,-0.036803,-0.036803,0.009930,0.000000,0.0,0.0043,-0.664644,0.657765,0.342235,-0.335356,0.000000,0.000000
1998-05-31,0.026373,0.026373,-0.011149,9.600395,0.0,0.0040,0.329647,0.670353,0.000000,-0.668927,0.000000,-0.331073
1998-06-30,-0.097781,-0.097781,0.024779,8.520072,0.0,0.0041,0.337712,0.662288,0.000000,-0.664425,0.000000,-0.335575
1998-07-31,0.054607,0.054607,-0.001774,8.401538,0.0,0.0040,0.329999,0.670001,0.000000,-0.668097,0.000000,-0.331903
1998-08-31,-0.104964,-0.104964,-0.133143,7.225777,0.0,0.0043,0.658421,0.341579,0.000000,-0.664423,0.000000,-0.335577
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.006211,-0.006211,0.043488,0.479122,0.0,0.0034,-0.557765,0.112298,-0.111565,0.547688,0.340014,-0.330670
2025-07-31,-0.002088,-0.002088,0.013121,0.478295,0.0,0.0034,-0.553889,0.111846,-0.109470,0.563054,0.325100,-0.336641
2025-08-31,-0.025726,-0.025726,0.026408,1.388585,0.0,0.0038,0.114636,-0.557326,-0.334319,0.331204,0.554160,-0.108355


# Portfolio Statistics

In [42]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,8.6789,-0.1337
Arithm Avg Xs Return,6.6559,-2.1567
Std Xs Returns,15.5029,13.3055
Sharpe Arithmetic,0.4293,-0.1621
Geom Avg Total Return,7.7266,-1.0645
Geom Avg Xs Return,5.6865,-3.1046
Sharpe Geometric,0.3668,-0.2333
Min Xs Return,-19.0140,-27.8689
Max Xs Return,12.8184,10.2359
Skewness,-0.6149,-1.9465
